In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import os

# Set random seed for reproducibility
tf.random.set_seed(42)

# Model parameters
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32
NUM_CLASSES = 4
EPOCHS = 20

def create_datasets(train_dir, valid_dir):
    # Data augmentation layer
    data_augmentation = tf.keras.Sequential([
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.2),
        layers.RandomZoom(0.2),
    ])
    
    # Load training dataset
    train_ds = tf.keras.utils.image_dataset_from_directory(
        train_dir,
        validation_split=None,
        seed=42,
        image_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        label_mode='categorical'
    )
    
    # Load validation dataset
    valid_ds = tf.keras.utils.image_dataset_from_directory(
        valid_dir,
        validation_split=None,
        seed=42,
        image_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        label_mode='categorical'
    )
    
    # Configure datasets for performance
    AUTOTUNE = tf.data.AUTOTUNE
    train_ds = train_ds.map(
        lambda x, y: (data_augmentation(x, training=True), y),
        num_parallel_calls=AUTOTUNE
    )
    
    # Normalize pixel values
    normalization_layer = layers.Rescaling(1./255)
    train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
    valid_ds = valid_ds.map(lambda x, y: (normalization_layer(x), y))
    
    # Use buffered prefetching
    train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
    valid_ds = valid_ds.prefetch(buffer_size=AUTOTUNE)
    
    return train_ds, valid_ds

def create_model():
    model = models.Sequential([
        # First Convolutional Block
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),
        
        # Second Convolutional Block
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),
        
        # Third Convolutional Block
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),
        
        # Fourth Convolutional Block
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),
        
        # Flatten and Dense Layers
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    
    return model

def train_model(train_dir, valid_dir):
    # Create datasets
    train_ds, valid_ds = create_datasets(train_dir, valid_dir)
    
    # Create and compile the model
    model = create_model()
    
    # Compile with mixed precision for faster training
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Add callbacks
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=2,
            min_lr=1e-6
        )
    ]
    
    # Train the model
    history = model.fit(
        train_ds,
        epochs=EPOCHS,
        validation_data=valid_ds,
        callbacks=callbacks
    )
    
    return model, history

def plot_training_history(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot accuracy
    ax1.plot(history.history['accuracy'])
    ax1.plot(history.history['val_accuracy'])
    ax1.set_title('Model Accuracy')
    ax1.set_ylabel('Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.legend(['Train', 'Validation'])
    
    # Plot loss
    ax2.plot(history.history['loss'])
    ax2.plot(history.history['val_loss'])
    ax2.set_title('Model Loss')
    ax2.set_ylabel('Loss')
    ax2.set_xlabel('Epoch')
    ax2.legend(['Train', 'Validation'])
    
    plt.show()

# Function to make predictions on new images
def predict_image(model, image_path):
    img = tf.keras.utils.load_img(
        image_path,
        target_size=(IMG_HEIGHT, IMG_WIDTH)
    )
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    img_array = img_array / 255.0
    
    predictions = model.predict(img_array)
    class_names = ['cow', 'elephant', 'horse', 'spider']
    predicted_class = class_names[tf.argmax(predictions[0])]
    confidence = tf.reduce_max(predictions[0])
    
    return predicted_class, confidence

# Example usage
if __name__ == "__main__":
    train_dir = "Train"
    valid_dir = "Valid"
    
    # Train the model
    model, history = train_model(train_dir, valid_dir)
    
    # Plot training history
    plot_training_history(history)
    
    # Save the model
    model.save('animal_classifier.h5')
    
    # Example of making a prediction
    # test_image_path = "path_to_test_image.jpg"
    # predicted_class, confidence = predict_image(model, test_image_path)
    # print(f"Predicted class: {predicted_class} with confidence: {confidence:.2f}")

Found 3997 files belonging to 4 classes.
Found 786 files belonging to 4 classes.
Epoch 1/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 124s 965ms/step - accuracy: 0.4407 - loss: 1.9292 - val_accuracy: 0.2443 - val_loss: 4.5649 - learning_rate: 0.0010
Epoch 2/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 119s 946ms/step - accuracy: 0.5312 - loss: 1.1947 - val_accuracy: 0.2468 - val_loss: 3.8665 - learning_rate: 0.0010
Epoch 3/20
110/125 ━━━━━━━━━━━━━━━━━━━━ 13s 928ms/step - accuracy: 0.6217 - loss: 1.0082